In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE156309'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of 61 FFPE tissues of DLBCL patients at high-risk (aaIPI 2 or 3)"
!Series_summary	"Current staging classifications do not accurately predict the benefit of high-dose chemotherapy (HDC) with autologous stem-cell transplantation (ASCT) for patients with diffuse large B-cell lymphoma (DLBCL) at high risk (age-adjusted International Index [aaIPI] score 2 or 3), who have achieved first complete remission after R-CHOP (rituximab, cyclophosphamide, vincristine, doxorubicin, and prednisone) treatment. We aim to construct a genetic prognostic model for improving individualized risk stratification and response prediction for HDC/ASCT therapy. We identified differentially expressed mRNAs associated with relapse of DLBCL."
!Series_overall_design	"Affymetrix Human U133 Plus 2.0 microarrays (ThermoFisher Scientific, Waltham, MA, USA) identified differentially expressed mRNAs between 34 relapse and 27 relapse-free DLBCL patients."
Sample Characte

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Affymetrix Human U133 Plus 2.0 microarrays indicates gene expression data is present

# Identify available variables
sample_characteristics_dict = {
    0: ['age: 37', 'age: 32', 'age: 35', 'age: 38', 'age: 26', 'age: 65', 'age: 36', 'age: 58', 'age: 19', 'age: 57', 'age: 55', 'age: 51', 'age: 30', 'age: 56', 'age: 29', 'age: 54', 'age: 27', 'age: 53', 'age: 39', 'age: 60', 'age: 33', 'age: 47', 'age: 34', 'age: 45', 'age: 31', 'age: 59', 'age: 25', 'age: 23', 'age: 52'],
    1: ['tissue: lymph node biopsy or puncture'],
    2: ['disease: Diffuse large B-cell lymphoma (DLBCL)'],
    3: ['disease status: relapse-free', 'disease status: relapse'],
    4: ['age-adjusted international index [aaipi] score: 2 or 3']
}

# Evaluate availability for 'X-Linked_Lymphoproliferative_Syndrome' (trait data)
trait_row = None  # Set to None as the dataset does not reference 'X-Linked_Lymphoproliferative_Syndrome'

# Evaluate availability for 'age'
if len(set(sample_characteristics_dict[0])) > 1:
    age_row = 0
else:
    age_row = None

# Gender information is not present in the dataset, so no need for gender conversion or evaluation
gender_row = None

# Define conversion functions
def convert_trait(value):
    return None  # There is no trait data available in the current context

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except (IndexError, ValueError):
        return None

# Save metadata
save_cohort_info('GSE156309', './preprocessed/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)


A new JSON file was created at: ./preprocessed/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json
